# Homework 1

### Q1

step1: 先讀取資料，再標準化所有feature values，觀察訓練與測試資料的大小。  
step2: 完成myknn_regressor的class：暴力法的演算法是先算出每筆測試資料與各個訓練資料的距離，然後找出距離最近的k個鄰居，再依照不同類型算出Y_train的平均值。  
step3: 計算實際值與預測值的RMSE。

In [78]:
import pickle
from sklearn import preprocessing
import numpy as np

In [79]:
# Load data
with open('msd_data1.pickle', 'rb') as fh1:
    msd_data = pickle.load(fh1)

doscaling = 1

if (doscaling == 1):
    xscaler = preprocessing.StandardScaler().fit(msd_data['X_train'])
    #standardize feature values
    X_train = xscaler.transform(msd_data['X_train'])
    X_test = xscaler.transform(msd_data['X_test'])
else:
    X_train = msd_data['X_train']
    X_test = msd_data['X_test']

Y_train = msd_data['Y_train']
Y_test = msd_data['Y_test']

In [80]:
# size of data
print(f'train_size: {X_train.shape[0]}')
print(f'test_size: {X_test.shape[0]}')

train_size: 5000
test_size: 3000


### Q1.1

In [91]:
class myknn_regressor():
    def __init__(self, n_neighbors = 10, mean_type = "equal_weight"):
        """mean_type can be equal_weight or remove_outliers.
                              equal_weight use the same weight for all neighbors.
                              remove_outliers remove neighbors out in [Q1 - 1.5 IQR, Q3 + 1.5IQR]."""
        ### Add your code here ###  
        '''save parameters'''   
        self.n_neighbors = n_neighbors 
        self.mean_type = mean_type

    def fit(self, x_train, y_train):
        ### Add your code here ###
        '''save training data'''
        self.x_train = x_train
        self.y_train = y_train
         

    def predict(self, x_test):
        """use remove_outliers only if k>=10"""
         ### Add your code here ###
        if self.n_neighbors < 10:
            self.mean_type = "equal_weight"
        
        
        self.x_test = x_test
        n_train = self.x_train.shape[0]
        n_test = self.x_test.shape[0]
        y_pred = []
        
        for i in range(n_test): # for every x_test
            dist = []
            for j in range(n_train): # for every x_train
                # Euclidean distance between x_test's ith row and x_train's jth row
                d = np.linalg.norm(self.x_test[i,:] - self.x_train[j,:])
                dist.append([d, self.y_train[j]])
            
            sort_dist = sorted(dist, key = lambda l: l[0]) # sort y_train by distance in ascending order


            # k nearest neighbors' y_train
            k_neighbors = []
            for k in range(self.n_neighbors):
                k_neighbors.append(sort_dist[k][1]) 

            
            if self.mean_type == "equal_weight":
                y_pred.append(np.mean(k_neighbors))
            else: # remove_outliers
                q1 = np.quantile(k_neighbors,0.25) 
                q3 = np.quantile(k_neighbors,0.75)
                iqr = q3 - q1
                
                new_neighbors = []
                for k in range(len(k_neighbors)):
                    y = k_neighbors[k]
                    if y >= (q1 - (1.5 * iqr)) and y <= (q3 + (1.5 * iqr)):
                        new_neighbors.append(y)

                y_pred.append(np.mean(new_neighbors))

        return y_pred

### Q1.2

In [92]:
# use myknn_regressor
myknn = myknn_regressor(20, "equal_weight")
myknn.fit(X_train, Y_train)
ypred = myknn.predict(X_test)

In [93]:
# count RMSE
actual = list(Y_test)
predicted = ypred
square = np.square(np.subtract(actual, predicted))
MSE = square.mean()
RMSE = np.sqrt(MSE)

print(f'RMSE: {RMSE}')

RMSE: 10.25126451549596


In [94]:
# print first 20 predictions
print("Q1.2 | first 20 predictions:")
print(ypred[:20])

Q1.2 | first 20 predictions:
[1993.35, 1993.8, 2000.65, 1991.5, 1992.8, 1998.5, 1988.1, 1991.65, 2002.25, 2003.0, 2000.5, 1998.65, 1995.55, 1997.2, 1995.05, 1997.4, 1992.15, 2000.45, 2003.2, 1995.75]


### Q1.3

In [95]:
# use myknn_regressor
myknn = myknn_regressor(20, "remove_outliers")
myknn.fit(X_train, Y_train)
ypred = myknn.predict(X_test)

In [96]:
# count RMSE
actual = list(Y_test)
predicted = ypred
square = np.square(np.subtract(actual, predicted))
MSE = square.mean()
RMSE = np.sqrt(MSE)

print(f'RMSE: {RMSE}')

RMSE: 10.212572466080376


In [97]:
# print first 20 predictions
print("Q1.3 | first 20 predictions:")
print(ypred[:20])

Q1.3 | first 20 predictions:
[1993.35, 1993.8, 2000.65, 1992.7368421052631, 1992.8, 2000.0, 1988.1, 1991.65, 2002.25, 2003.9473684210527, 2000.5, 2000.9444444444443, 1995.55, 1997.2, 1998.611111111111, 1997.4, 1992.15, 2003.8333333333333, 2003.2, 1995.75]


### Q2